In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
import copy
import datetime
import json
import os
import scipy.spatial
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [2]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
with open('./data/western_north_america/command.json', 'r') as f:
    command = json.load(f)
station = pd.read_csv(command["station_file_name"])
segment = pd.read_csv(command["segment_file_name"])
block = pd.read_csv(command["block_file_name"])

In [3]:
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)

In [ ]:
# Trying a direct port of BlockLabel from Blocks
# function [S, b, st] = BlockLabel(s, b, st)

#     % Split any prime meridian-crossing segments
#     split = segmeridian(s);
#     nseg = numel(split.lon1);

#     % make sure western vertex is the start point
#     [S, i] = OrderEndpointsSphere(split);
#     [S.x1, S.y1, S.z1] = sph2cart(DegToRad(S.lon1(:)), DegToRad(S.lat1(:)), 6371);
#     [S.x2, S.y2, S.z2] = sph2cart(DegToRad(S.lon2(:)), DegToRad(S.lat2(:)), 6371);
#     sego = [S.lon1(:) S.lon2(:)];
#     sega = [S.lat1(:) S.lat2(:)];
#     segx = [S.x1(:) S.x2(:)];
#     segy = [S.y1(:) S.y2(:)]; segy(abs(segy) < 1e-6) = 0;
#     segz = [S.z1(:) S.z2(:)];
#     i = (i-1)*nseg + repmat((1:nseg)', 1, 2);

#     % make sure there are no hanging segments
#     allc = [segx(:) segy(:) segz(:)];
#     %allc = [[s.x1(:) s.y1(:) s.z1(:)]; [s.x2(:) s.y2(:) s.z2(:)]];
#     [~, i1] = unique(allc, 'rows', 'first');
#     [~, i2] = unique(allc, 'rows', 'last');
#     if isempty(~find(i2-i1, 1))
#         fprintf(1, '*** All blocks are not closed! ***\n');
#         %else
#         %fprintf(1, 'No hanging segments found');
#     end

#     % Carry out a few operations on all segments

#     % Find unique points and indices to them
#     [~, ~, ui] = unique(allc, 'rows', 'first');
#     us = ui(1:nseg); ue = ui(nseg+1:end);

#     % Calculate the azimuth of each fault segment
#     % Using atan instead of azimuth because azimuth breaks down for very long segments
#     % az1 = rad2deg(atan2(segx(:, 1) - segx(:, 2), segy(:, 1) - segy(:, 2)));
#     % az2 = rad2deg(atan2(segx(:, 2) - segx(:, 1), segy(:, 2) - segy(:, 1)));

#     % Make local azimuths for every endpoint. The problem is indeed for really long segments;
#     % the junction is essentially approached along an azimuth that doesn't sensibly fall between
#     % the "exit" azimuths. So, for each end point, create a new test point 1 degree away along
#     % the great circle, but because it's closer, the great circle azimuth will be "local" and
#     % therefore makes the junction decision more sensibly.
#     %
#     % An example problem segment is the northern boundary of North America in the Blocks release
#     % segment file. Its azimuth eastward, away from the northern Pacific Ocean, is about 38, but
#     % westward away from Greenland, it's 315. This ends up affecting the Pacific and exterior blocks.

#     % Local azimuths originating from endpoint 1
#     az2 = sphereazimuth(sego(:, 1), sega(:, 1), sego(:, 2), sega(:, 2)); % Whole segment azimuth
#     [tlo2, tla2] = gcpoint(sego(:, 1), sega(:, 1), az2, 1);   % Local test point coordinates
#     az21 = sphereazimuth(sego(:, 1), sega(:, 1), tlo2, tla2); % Toward test point
#     az22 = sphereazimuth(tlo2, tla2, sego(:, 1), sega(:, 1)); % From test point

#     % Local azimuths originating from endpoint 2
#     az1 = sphereazimuth(sego(:, 2), sega(:, 2), sego(:, 1), sega(:, 1)); % Local test point coordinates
#     [tlo1, tla1] = gcpoint(sego(:, 2), sega(:, 2), az1, 1);   % Whole segment azimuth
#     az11 = sphereazimuth(sego(:, 2), sega(:, 2), tlo1, tla1); % Toward test point
#     az12 = sphereazimuth(tlo1, tla1, sego(:, 2), sega(:, 2)); % From test point
#     saz = [az21 az11];
#     eaz = [az22 az12];
#     saz(saz < 0) = saz(saz < 0) + 360;
#     eaz(eaz < 0) = eaz(eaz < 0) + 360;

#     % Declare array to store polygon segment indices
#     poly_ver = zeros(1, nseg);
#     trav_ord = poly_ver;
#     seg_poly_ver = zeros(nseg);
#     seg_trav_ord = seg_poly_ver;

#     for i = 1:nseg
#         % establish starting coordinates
#         cs = i;      % current segment start
#         cp = us(i);  % current point: start point of the current segment
#         se = 1;      % flag indicating that it's looking towards ending point
#         starti = cs; % index of the starting point
#         seg_cnt = 1;

#         clear poly_vec trav_ord

#         while 1
#             matchss = (us == cp); % starts matching current
#             matchss(cs) = 0;
#             matchss = find(matchss);
#             matches = (ue == cp); % ends matching current
#             matches(cs) = 0;
#             matches = find(matches);

#             match = [matchss; matches];

#             % If it's a multiple intersection, find which path to take
#             if numel(match) > 1
#                 daz = saz(cs, se) - [eaz(matchss, 1); saz(matches, 1)];
#                 daz(abs(daz) > 180) = daz(abs(daz) > 180) - sign(daz(abs(daz) > 180))*360;
#                 [~, mi] = max(daz);
#             else
#                 mi = 1;
#             end
#             match = match(mi);

#             % Determine the new starting point
#             cs = match; % current index
#             if mi <= numel(matchss) % if the index is a start-current match
#                 cp = ue(cs); % the new point is the match's ending point
#                 se = 2; % looking towards the start point
#             else
#                 cp = us(cs); % otherwise it's the match's starting point
#                 se = 1; % looking towards the end point
#             end

#             % Prevent endless loops
#             if seg_cnt > nseg
#                 disp(sprintf('Cannot close block starting with segment: %s', s.name(starti, :)))
#                 break;
#             end

#             % Break upon returning to the starting segment
#             if match == starti && seg_cnt > 1
#                 seg_cnt = 1;
#                 poly_vec = [poly_vec, starti];
#                 trav_ord = [trav_ord, se];
#                 break;
#             else
#                 poly_vec(seg_cnt) = cs;
#                 trav_ord(seg_cnt) = se;
#                 seg_cnt = seg_cnt + 1;
#             end
#         end

#         %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#         %%  Put poly_vec into seg_poly_ver                     %%
#         %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#         seg_poly_ver(i, 1:length(poly_vec)) = poly_vec;
#         seg_trav_ord(i, 1:length(trav_ord)) = trav_ord;
#     end

#     % Determine the unique block polygons
#     [so, blockrows] = unique(sort(seg_poly_ver, 2), 'rows');
#     seg_poly_ver = seg_poly_ver(blockrows, :);
#     seg_trav_ord = seg_trav_ord(blockrows, :);
#     z = ~so;
#     so(z) = NaN;
#     so = sort(so, 2);
#     [~, blockrows] = sortrows(so);
#     seg_poly_ver = seg_poly_ver(blockrows, :);
#     seg_trav_ord = seg_trav_ord(blockrows, :);

#     % Determine number of blocks
#     nblock = size(seg_poly_ver, 1);

#     % Calculate block area and label each block
#     barea = zeros(nblock, 1);
#     alabel = zeros(nblock, 1);
#     ext = 0;

#     el = zeros(nseg, 1);
#     wl = el;
#     stl = zeros(numel(st.lon), 1);
#     dLon = 1e-6;

#     % Convert interior points to Cartesian
#     [bix, biy, biz] = sph2cart(DegToRad(b.interiorLon), DegToRad(b.interiorLat), 6371);

#     for i = 1:nblock
#         % Take block coordinates from the traversal order matrix
#         sib = seg_poly_ver(i, (seg_poly_ver(i, :) ~= 0)); % segments in block
#         ooc = seg_trav_ord(i, (seg_trav_ord(i, :) ~= 0)); % order in which the segments are traversed
#         cind = (ooc-1)*nseg + sib; % convert index pairs to linear index
#         bco = sego(cind)';
#         bca = sega(cind)';
#         % Test which interior points lie within the current block
#         [bin, saz] = inpolygonsphere(b.interiorLon, b.interiorLat, bco, bca);
#         if sum(bin) > 1
#            [bin, saz] = inpolygonsphere(b.interiorLon, b.interiorLat, bco, bca, saz-180);
#         end
#         barea(i) = polyareasphere(bco, bca);
#         % Now test the segments for labeling east and west sides
#         testlon = S.midLon(sib) + dLon; % perturbed midpoint longitude
#         cin = inpolygonsphere(testlon, S.midLat(sib), bco, bca, saz); % test to see which perturbed coordinates lie within the current block
#         % Now test the station coordinates for block identification
#         stin = inpolygonsphere(st.lon, st.lat, bco, bca, saz);
#         if sum(bin) > 1 % exterior block or error
#             if abs(barea(i)) == max(abs(barea)) && ext == 0 % if the area is the largest and exterior hasn't yet been assigned
#                 alabel(find(~bin)) = i; % ...assign this block as the exterior
#                 ext = i; % and specifically declare the exterior label
#             elseif ext > 0
#                 disp('Interior points do not uniquely define blocks!')
#                 break;
#             end
#         else % if there is only one interior point within the segment polygon (i.e., all other blocks)...
#             alabel(find(bin)) = i; % assign that block associate label to the current block
#             el(sib(cin > 0)) = i; % segments within the polygon are assigned this block as their east label
#             wl(sib(cin == 0)) = i; % those that don't are assigned this block as their west label
#             stl(stin > 0) = i; % associate stations with the block
#         end

#         % Add ordered polygons to the blocks structure
#         b.orderLon{i} = [bco(:); bco(1)];
#         b.orderLat{i} = [bca(:); bca(1)];
#     end

#     if ext == 0 % If exterior block is unassigned, 
#        if length(b.interiorLon) <= 2 % Special case for a single block
#           ext = 2;
#           alabel = [1 2];
#           b.orderLon{2} = b.orderLon{1};
#           b.orderLat{2} = b.orderLat{1};
#        else
#           ext = setdiff(1:nblock, alabel); % Special case for a north pole block
#        end 
#     end
    
#     if isempty(ext) % Final test if exterior block has not been labeled; label it by area
#        [~, ext] = max(abs(barea));
#     end

#     % treat exterior block segment labels - set exterior block for yet undefined segment labels
#     el(el == 0) = ext;
#     wl(wl == 0) = ext;
#     % treat exterior block stations
#     stl(stl == 0) = ext;

#     % Final outputs
#     S.eastLabel = el;
#     S.westLabel = wl;

#     [st.blockLabel, st.blockLabelUnused] = deal(stl);

#     % Reorder block properties
#     alabel(alabel == 0) = ext;
#     b = BlockReorder(alabel, b);
#     b.associateLabel = alabel;
#     b.exteriorBlockLabel = ext;


In [4]:
np_segments = np.zeros((len(segment), 2, 2))
np_segments[:, 0, 0] = segment.lon1.to_numpy()
np_segments[:, 1, 0] = segment.lon2.to_numpy()
np_segments[:, 0, 1] = segment.lat1.to_numpy()
np_segments[:, 1, 1] = segment.lat2.to_numpy()

In [5]:
# De-duplicate the vertices and build an ptr_edge array
all_vertices = np_segments.reshape((-1, 2))
tree = scipy.spatial.KDTree(all_vertices, leafsize=1000)
duplicates = tree.query_ball_point(all_vertices, 1e-8)

ptr_edges = []
dedup_vertices = []
original_to_new = dict()
for i in range(np_segments.shape[0]):
    v1_idx = duplicates[2 * i][0]
    v2_idx = duplicates[2 * i + 1][0]
    if v1_idx == 2 * i:
        original_to_new[2 * i] = len(dedup_vertices)
        dedup_vertices.append(np_segments[i][0])
    if v2_idx == 2 * i + 1:
        original_to_new[2 * i + 1] = len(dedup_vertices)
        dedup_vertices.append(np_segments[i][1])
    ptr_edges.append((original_to_new[v1_idx], original_to_new[v2_idx]))

np_dedup_vertices = np.array(dedup_vertices)

# Check that the vertices are unique up to 1e-8 now.
new_tree = scipy.spatial.KDTree(np_dedup_vertices)
np.all([v[0] == i for i,v in enumerate(new_tree.query_ball_point(np_dedup_vertices, 1e-8))])

# Build the graph
G = nx.Graph()
for e in ptr_edges:
    G.add_edge(*e)

# Check that it is planar and build the PlanarEmbedding data structure.
is_planar, planar_embedding = nx.check_planarity(G)

# Extract each polygon. 
counted_half_edges = set()
polygons = []
for component in nx.connected_components(planar_embedding):
    for v in component:
        for w in planar_embedding.neighbors_cw_order(v):
            if (v, w) not in counted_half_edges:
                polygons.append(planar_embedding.traverse_face(v, w, counted_half_edges))

print("Found " + str(len(polygons)) + " closed polygons")

# Map polygons boundaries back to 2 labesl for each block
n_vertices = np_dedup_vertices.shape[0]
n_edges = len(ptr_edges)

vertex_to_polygon = [[] for i in range(n_vertices)] 
for p_idx, p in enumerate(polygons):
    for v_idx in p:
        vertex_to_polygon[v_idx].append(p_idx)

edge_to_polygon = []
for e in ptr_edges:
    polys_v0 = vertex_to_polygon[e[0]]
    polys_v1 = vertex_to_polygon[e[1]]
    polys_edge = [p_idx for p_idx in polys_v0 if p_idx in polys_v1] # Intersection of lists
    edge_to_polygon.append(polys_edge)

segment["initial_label1"] = np.array(edge_to_polygon)[:, 0]
segment["initial_label2"] = np.array(edge_to_polygon)[:, 1]

Found 6 closed polygons


In [6]:
polygon_count = 0
for p in polygons:
    verts_loop = np.concatenate([np_dedup_vertices[p], np_dedup_vertices[p[0]][None,:]])
    plt.figure()
    plt.plot(verts_loop[:, 0], verts_loop[:, 1])
    plt.title(str(polygon_count))
    plt.show()
    polygon_count += 1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
longitude_east_offset = 1e-3
unprocessed_value = 99
correct_east_labels = unprocessed_value * np.ones(len(segment))
correct_west_labels = unprocessed_value * np.ones(len(segment))

for current_block_label in range(len(polygons)):
    current_block_idx = np.union1d(segment.index[segment.initial_label1 == current_block_label],
                                   segment.index[segment.initial_label2 == current_block_label])
    segment_current_block = segment.iloc[current_block_idx]
    polygon_vertex_idx = polygons[current_block_label]
    vertices_current_block = np.concatenate([np_dedup_vertices[polygon_vertex_idx], np_dedup_vertices[polygon_vertex_idx[0]][None,:]])
    is_midpoint_inpolygon = celeri.inpolygon(segment_current_block.mid_lon_plate_carree.values + longitude_east_offset,
                                segment_current_block.mid_lat_plate_carree.values,
                                vertices_current_block[:, 0], vertices_current_block[:, 1])
    correct_east_labels[current_block_idx[np.where(is_midpoint_inpolygon == True)]] = current_block_label
    correct_west_labels[current_block_idx[np.where(is_midpoint_inpolygon == False)]] = current_block_label

# Find unprocessed indices
unprocessed_idx = np.union1d(np.where(correct_east_labels == unprocessed_value),
                             np.where(correct_west_labels == unprocessed_value))

# plt.figure()
# plt.title("Segment indices")
# for i in range(len(segment)):
#     plt.plot([segment.lon1.values[i], segment.lon2.values[i]],
#              [segment.lat1.values[i], segment.lat2.values[i]], "-k")
#     plt.text(segment.mid_lon_plate_carree.values[i] - 0.25,
#              segment.mid_lat_plate_carree.values[i],
#              str(i), fontsize=6)

plt.figure()
plt.title("Initial labels")
for i in range(len(segment)):
    plt.plot([segment.lon1.values[i], segment.lon2.values[i]],
             [segment.lat1.values[i], segment.lat2.values[i]], "-k")
    plt.text(segment.mid_lon_plate_carree.values[i] - 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(segment.initial_label1[i]), fontsize=16)
    plt.text(segment.mid_lon_plate_carree.values[i] + 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(segment.initial_label2[i]), fontsize=16)

plt.figure()
plt.title("Attempt at east and west labels")
for i in range(len(segment)):
    plt.plot([segment.lon1.values[i], segment.lon2.values[i]],
             [segment.lat1.values[i], segment.lat2.values[i]], "-k")
    plt.text(segment.mid_lon_plate_carree.values[i] - 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(correct_west_labels[i]), fontsize=16)
    plt.text(segment.mid_lon_plate_carree.values[i] + 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(correct_east_labels[i]), fontsize=16)

for i in range(len(unprocessed_idx)):
    plt.plot([segment.lon1.values[unprocessed_idx[i]], segment.lon2.values[unprocessed_idx[i]]],
             [segment.lat1.values[unprocessed_idx[i]], segment.lat2.values[unprocessed_idx[i]]], "-r")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
edge_to_polygon

[[0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [0, 4],
 [0, 4],
 [0, 4],
 [0, 4],
 [0, 4],
 [0, 4],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [0, 2],
 [2, 3],
 [0, 4],
 [0, 4],
 [2, 4],
 [2, 4],
 [2, 4],
 [2, 4],
 [0, 4],
 [0, 4],
 [2, 4],
 [2, 4],
 [2, 4],
 [2, 4],
 [0, 4],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [2, 3],
 [2, 3],
 [0, 5],
 [0, 5],
 [0, 4],
 [0, 4],
 [4, 5],
 [4, 5],
 [4, 5],
 [4, 5],
 [4, 5],
 [4, 5],
 [2, 4],
 [2, 4],
 [2, 4],
 [4, 5],
 [2, 3],
 [2, 3],
 [2, 3],
 [2, 3],
 [2, 3],
 [2, 3],
 [2, 3],
 [0, 2],
 [0, 2],
 [0, 3],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 2]]